In [1]:
import numpy as np
import pandas as pd

In [2]:
test = pd.read_csv('../data/X_test.csv')

In [3]:
import sys
sys.path.append('../tools/')

In [4]:
from model_manager import model_manager as mm

In [5]:
svd = mm('svd_model', '../models/')
user_encoder = mm('user_encoder', '../models/')
item_encoder = mm('item_encoder', '../models/')
tv = mm('tv', '../models/')
xgr = mm('xgr', '../models/')
lr = mm('Linear_regression', '../models/')

In [6]:
models = {'tv': tv, 'user': user_encoder, 'item': item_encoder, 'svd': svd, 'xgr': xgr, 'lr': lr}

In [7]:
for _, model in models.items():
    model.load()

Done
Done
Done
Done
Done
Done


In [8]:
models = {name: model.model for name, model in models.items()}

In [9]:
test['sentiment'] = test.sentiment.apply(eval)
test['sum_sentiment'] = test.sum_sentiment.apply(eval)

In [10]:
test['sentiment'] = test.sentiment.apply(np.array)
test['sum_sentiment'] = test.sum_sentiment.apply(np.array)

In [11]:
unknown = test[(test.UserId.isin({'A39PU8IUVZ6J8T', 'AADQXONRGI10Z'})) | (test.ProductId.isin({'B00006L942', 'B001D5DQIY'}))].reset_index(drop = True)
known = test[~((test.UserId.isin({'A39PU8IUVZ6J8T', 'AADQXONRGI10Z'})) | (test.ProductId.isin({'B00006L942', 'B001D5DQIY'})))].reset_index(drop = True)

In [12]:
## user item text helpfulnum helpfulden senti sum_senti

In [13]:
from text_preprocessor import preprocessor as pps

In [14]:
p = pps()

In [15]:
sk = list(known[['Text']].values)

In [16]:
sk = p.fit_transform(sk)

100%|██████████| 299980/299980 [15:37<00:00, 320.11it/s]


In [17]:
msk = np.concatenate([known[['HelpfulnessNumerator', 'HelpfulnessDenominator']].values, np.stack(known.values[:, -2]), np.stack(known.values[:, -1])],axis=1)
musk = np.concatenate([unknown[['HelpfulnessNumerator', 'HelpfulnessDenominator']].values, np.stack(unknown.values[:, -2]), np.stack(unknown.values[:, -1])],axis=1)

In [18]:
np.savez('../data/known', msk)
np.savez('../data/unknown', musk)

In [19]:
known['UserId'] = models['user'].transform(known['UserId'])
known['ProductId'] = models['item'].transform(known['ProductId'])

In [20]:
from scipy import sparse

In [21]:
msk = sparse.hstack([sparse.csr_matrix(known[['Id']].values), sparse.csr_matrix(models['svd'].pu[known.UserId.values]), sparse.csr_matrix(models['svd'].qi[known.ProductId.values]), models['tv'].transform(sk), sparse.csr_matrix(known[['HelpfulnessNumerator', 'HelpfulnessDenominator']].values), sparse.csr_matrix(np.stack(known.values[:, -2])), sparse.csr_matrix(np.stack(known.values[:, -1]))])

In [22]:
unknown1 = unknown[(unknown.UserId.isin({'A39PU8IUVZ6J8T', 'AADQXONRGI10Z'}))].reset_index(drop = True)
unknown2 = unknown[(unknown.ProductId.isin({'B00006L942', 'B001D5DQIY'}))].reset_index(drop = True)

In [23]:
unknown2['UserId'] = models['user'].transform(unknown2['UserId'])
unknown1['ProductId'] = models['item'].transform(unknown1['ProductId'])

In [24]:
nMat = np.full([10, 20], np.nan)

In [25]:
suk1 = list(unknown1[['Text']].values)
suk1 = p.fit_transform(suk1)

100%|██████████| 10/10 [00:00<00:00, 1119.77it/s]


In [26]:
suk2 = list(unknown2[['Text']].values)
suk2 = p.fit_transform(suk2)

100%|██████████| 10/10 [00:00<00:00, 359.59it/s]


In [27]:
models['svd'].qi[unknown1.ProductId.values].shape

(10, 20)

In [28]:
models['svd'].pu[unknown2.UserId.values].shape

(10, 20)

In [29]:
musk1 = sparse.hstack([sparse.csr_matrix(unknown1[['Id']].values), sparse.csr_matrix(nMat), sparse.csr_matrix(models['svd'].qi[unknown1.ProductId.values]), models['tv'].transform(suk1), sparse.csr_matrix(unknown1[['HelpfulnessNumerator', 'HelpfulnessDenominator']].values), sparse.csr_matrix(np.stack(unknown1.values[:, -2])), sparse.csr_matrix(np.stack(unknown1.values[:, -1]))])

In [30]:
musk2 = sparse.hstack([sparse.csr_matrix(unknown2[['Id']].values), sparse.csr_matrix(models['svd'].pu[unknown2.UserId.values]), sparse.csr_matrix(nMat), models['tv'].transform(suk2), sparse.csr_matrix(unknown2[['HelpfulnessNumerator', 'HelpfulnessDenominator']].values), sparse.csr_matrix(np.stack(unknown2.values[:, -2])), sparse.csr_matrix(np.stack(unknown2.values[:, -1]))])

In [31]:
np.savez('../data/test_matrix', X = sparse.vstack([msk, musk1, musk2]))

In [32]:
sparse.vstack([msk, musk1, musk2]).shape

(300000, 446)